<a href="https://colab.research.google.com/github/fangyuan-ksgk/ChatBot_Experiments/blob/main/Latency_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. **Dependency & Installation**

In [1]:
%cd /content
!git clone -b dev https://github.com/camenduru/OpenVoice
%cd /content/OpenVoice

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/OpenVoice/resolve/main/checkpoints_1226.zip -d /content -o checkpoints_1226.zip
!unzip /content/checkpoints_1226.zip

!pip install -q gradio==3.50.2 langid faster-whisper whisper-timestamped unidecode eng-to-ipa pypinyin cn2an

import locale
# WorkAround
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding()
# !pip install sounddevice
!sudo apt-get install libportaudio2
!pip install sounddevice

# !python openvoice_app.py --share

/content
Cloning into 'OpenVoice'...
remote: Enumerating objects: 211, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 211 (delta 90), reused 85 (delta 85), pack-reused 103
Receiving objects: 100% (211/211), 2.28 MiB | 15.66 MiB/s, done.
Resolving deltas: 100% (94/94), done.
/content/OpenVoice
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121671 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd

2. **Mistral Model Loading** : mistral-7B-Instruct-v0.2

In [73]:
# Mistral inference
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Dirty Implementaion on-the-cloud
* Need to use my Alienware Gadget for this one

In [76]:
import time

import os
import torch
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter
from faster_whisper import WhisperModel

def text_to_text(user_text, llm=llm):
  messages = [
      {"role": "user", "content": "Hello."},
      {"role": "assistant", "content": "Hello."},
      {"role": "user", "content": user_text}
  ]

  start = time.time()
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  llm.to(device)

  generated_ids = llm.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  end = time.time()
  print('----'*10)
  print('Mistral 7B Processing Time: ', end-start)
  print(decoded[0])
  print('----'*10)

  text = decoded[0].split('[/INST]')[-1].replace('\n', '')
  return text

def text_to_speech(text, output_name='output_en_default'):
  start = time.time()
  ckpt_base = 'checkpoints/base_speakers/EN'
  ckpt_converter = 'checkpoints/converter'
  device = "cuda:0" if torch.cuda.is_available() else "cpu"
  output_dir = 'outputs'

  base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
  base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

  tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
  tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

  os.makedirs(output_dir, exist_ok=True)

  source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

  reference_speaker = 'resources/example_reference.mp3'
  target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

  save_path = f'{output_dir}/{output_name}.wav'

  # Run the base speaker tts
  # src_path = f'{output_dir}/tmp.wav'
  src_path = f'{output_dir}/Elon-Audio.wav'
  base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

  # Run the tone color converter
  encode_message = "@MyShell"
  tone_color_converter.convert(
      audio_src_path=src_path,
      src_se=source_se,
      tgt_se=target_se,
      output_path=save_path,
      message=encode_message)

  end = time.time()
  print('----'*10)
  print('OpenVoice Text2Speech Time: ', end-start)
  print('----'*10)

def speech_to_text(audio_name):
  start = time.time()
  model_size = "large-v3"
  # Run on GPU with FP16
  model = WhisperModel(model_size, device="cuda", compute_type="float16")
  segments, info = model.transcribe(f"outputs/{audio_name}.wav", beam_size=5)

  print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

  text = ''
  for segment in segments:
    text += segment.text
    print("Speech Time [%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
  end = time.time()
  print('----'*10)
  print('Faster-Whisper Speech2Text: ', end-start)
  print('----'*10)
  return text

A100 (80GB) Statistics:
* Faster-Whisper (Speech2Text): 4.98 s
* Mistral-7B (non-quantized): 9.8 s
* OpenVoice (Text2Speech): 5.22 s

Note that here the Time is completion time, in practice, latency is the time-till-first response. || My guess is some tricks are played in Kris's repo which reduces latency significantly.

In [77]:
text = speech_to_text('Elon-Audio') # MicroPhone input
response = text_to_text(text) # Mistral Inferece
speech = text_to_speech(response, 'response-audio') # Mistral Inferece

Detected language 'en' with probability 1.000000
Speech Time [0.00s -> 4.50s]  Elon Musk is a controversial figure and people's feelings towards him are quite diverse.
Speech Time [5.00s -> 9.32s]  Some view him as a visionary and an innovator, admiring his entrepreneurial spirit,
Speech Time [9.82s -> 14.06s]  his tenacity, and his ability to bring groundbreaking technologies to market.
Speech Time [14.62s -> 19.28s]  Others criticize him for his unconventional business practices, his sometimes erratic
Speech Time [19.28s -> 24.22s]  behavior, and his apparent disregard for conventional wisdom or established rules.
Speech Time [24.56s -> 29.36s]  Musk's public persona is often the subject of intense scrutiny, and there are those who
Speech Time [29.36s -> 32.50s]  appreciate his brashness and those who find it off-potting.
Speech Time [32.98s -> 37.40s]  His tweets, in particular, have been known to cause controversy, with some seeing them
Speech Time [37.40s -> 42.34s]  as a source o

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Speech Time [58.36s -> 63.52s]  One thing is certain Musk is a polarizing figure and his impact on the world of business.
Speech Time [64.02s -> 69.06s]  Technology and transportation is undeniable whether you view him as a visionary or a disruptor.
----------------------------------------
Faster-Whisper Speech2Text:  4.984947204589844
----------------------------------------
----------------------------------------
Mistral 7B Processing Time:  9.861910581588745
<s> [INST] Hello. [/INST]Hello.</s> [INST]  Elon Musk is a controversial figure and people's feelings towards him are quite diverse. Some view him as a visionary and an innovator, admiring his entrepreneurial spirit, his tenacity, and his ability to bring groundbreaking technologies to market. Others criticize him for his unconventional business practices, his sometimes erratic behavior, and his apparent disregard for conventional wisdom or established rules. Musk's public persona is often the subject of intense scrutiny, and t

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []
 > Text splitted to sentences.
You're absolutely right. Elon Musk is a complex and controversial figure who elicits strong reactions from people.
Some view him as a visionary entrepreneur who is pushing the boundaries of innovation and making groundbreaking advancements in various industries,
particularly in technology and transportation. Others see him as a disruptor who ignores traditional business practices and norms,
and whose unconventional methods and erratic behavior raise concerns. Musk's public persona is certainly a subject of intense scrutiny,
with his tweets being a particular point of scrutiny for some.
Some people find his tweets entertaining, while others view them as evidence of impulsivity or lack of professionalism.
Ultimately, people's opinions of Musk depend on their individual perspect